# Guide LLM to be as concise as possible

### Load Helper Functions and Import Libraries

In [1]:
import datetime
import json
import time
import os
import datetime
import json
import time
from openai import AzureOpenAI
from dotenv import load_dotenv
import json
import copy
import textwrap

# cost per token for GPT 4 32K
prompt_token_cost = 0.06/1000
generation_token_cost = 0.12/1000


# Load environment variables
load_dotenv()

def aoai_call(system_message,prompt,model):
    client = AzureOpenAI(
        api_version=os.getenv("API_VERSION"),
        azure_endpoint=os.getenv("AZURE_ENDPOINT"),
        api_key=os.getenv("API_KEY")
    )

    start_time = time.time()

    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt},
        ],
    )

    end_time = time.time()
    e2e_time = end_time - start_time

    result=json.loads(completion.model_dump_json(indent=2))
    prompt_tokens=result["usage"]["prompt_tokens"]
    completion_tokens=result["usage"]["completion_tokens"]
    completion_text=result["choices"][0]["message"]["content"]

    return result,prompt_tokens,completion_tokens,completion_text,e2e_time

model=os.getenv("MODELGPT432k")

# Read essay from a text file
with open('sales_report.txt', 'r') as f:
    sales_report = f.read()

### A: Naive summarisation, using the model's default verbosity

In [2]:
system_message="""
You are a helpful AI assistant.
"""
prompt=f"""
Document to summarise:
{sales_report}
Summarise this document. Include the type of game, the sales performance by region, the performance of the launch, and feedback about the launch.
"""

result,prompt_tokens,completion_tokens,completion_text,e2e_time=aoai_call(system_message,prompt,model)
print(f"Prompt Tokens: {prompt_tokens}")
print(f"Completion Tokens: {completion_tokens}")
print(f"Total cost: ${prompt_tokens*prompt_token_cost + completion_tokens*generation_token_cost:.4f}")
print(f"Time taken: {e2e_time:.2f} seconds")


Prompt Tokens: 1554
Completion Tokens: 277
Total cost: $0.1265
Time taken: 20.11 seconds


In [3]:
completion_text

"The document discusses the global launch of a flagship real-time strategy (RTS) video game with an immersive gameplay, strategic planning, dynamic environments and characters. The game's launch exceeded expectations, particularly in the North American markets where the game sold 1.5 million units against the projected one million units. This success was mirrored in its global sales, especially in the European and Asian markets, which sold 1.8 and 2.0 million units respectively. There was promising growth in the Latin American and African markets, despite lower sales numbers (0.5 and 0.3 million units respectively).\n\nUser feedback overall was a mixture of praise and critique. The game's mechanics and strategic elements received commendation, but the micro-transaction system, which involves the use of real money to buy in-game currency and expedite progress, was criticised for making the game too expensive and providing an unfair advantage to players who spend more.\n\nThe game's impr

### B: Summarisation with a focus on conciseness

In [4]:
system_message="""
You are a helpful AI assistant.
"""
prompt=f"""
Document to summarise:
{sales_report}
Summarise this document. Include the type of game, the sales performance by region, the performance of the launch, and feedback about the launch. Be as succint as possible, using as few words as possible.
"""

result,prompt_tokens,completion_tokens,completion_text,e2e_time=aoai_call(system_message,prompt,model)
print(f"Prompt Tokens: {prompt_tokens}")
print(f"Completion Tokens: {completion_tokens}")
print(f"Total cost: ${prompt_tokens*prompt_token_cost + completion_tokens*generation_token_cost:.4f}")
print(f"Time taken: {e2e_time:.2f} seconds")


Prompt Tokens: 1568
Completion Tokens: 135
Total cost: $0.1103
Time taken: 8.49 seconds


In [5]:
completion_text

"The document details the global launch of a real time strategy (RTS) video game. The North American launch was highly successful, with sales surpassing initial projections by 1.5 times. Similar success was witnessed in international markets, particularly in Europe and Asia with sales of 1.8 and 2.0 million units respectively. Despite lower sales numbers, potential was identified in the Latin American and African markets. However, user feedback showcased dissatisfaction with the in-game micro-transaction system, perceived as a 'pay-to-win' model. Consequently, future plans involve re-evaluating the micro-transaction system and developing strategies for better penetration into underperforming markets."

### C: Few shot prompting

In [6]:
system_message="""
You are a helpful AI assistant.
"""
prompt=f"""


You must use the below structure for your summary.
START_EXAMPLE:
Document:
Product Briefing Document
I. Product Overview
Product: "Battlefield Conquerors"
Battlefield Conquerors is an immersive, action-packed First-Person Shooter (FPS) game that thrusts players into a gritty, fast-paced world of strategic warfare. Skill, precision, and quick thinking are the keys to victory in this adrenaline-fueled gaming experience.
II. Sales Performance
Battlefield Conquerors has achieved robust sales across multiple global regions, effectively penetrating the gaming market.
North America (NA)
In North America, the game has resonated particularly well, with a total of 1.5 million units sold. This success can be attributed to its strategic marketing campaign and the region's affinity for the FPS genre.
Europe (EU)
Europe has emerged as the game's most successful region in terms of sales, with an impressive 2.0 million units sold. The game's realistic graphics, dynamic gameplay, and stimulating storylines have been lauded by European gamers.
Asia-Pacific (APAC)
In the APAC region, Battlefield Conquerors has sold 1.0 million units. This solid performance is a testament to its broad appeal and the successful localization of the game's content for these markets.
III. Financial Performance
Battlefield Conquerors has not only met its financial targets but exceeded them. The game has achieved a performance of 1.2 times its initial budget, demonstrating its profitability and the successful return on investment.
IV. Customer Feedback
Customer feedback for Battlefield Conquerors has been overwhelmingly positive. Players have praised the game's innovative mechanics, immersive environment, and challenging gameplay.
However, some concerns have been raised regarding the game's loading times. These issues have been acknowledged and are currently being addressed by the development team to ensure a seamless and uninterrupted gaming experience for all users moving forward.
In conclusion, Battlefield Conquerors is a successful and profitable product, demonstrating strong sales performance across multiple regions. The few areas of improvement identified are being addressed to ensure continued success and customer satisfaction.

Summary:
Product Type:
FPS

Sales:
NA: 1.5M
EU: 2.0M
APAC: 1.0M

Performance:
1.2 times budget.

Feedback:
Overall positive. Some concerns around loading times.
END_EXAMPLE

Document:
{sales_report}

Summary:
"""

result,prompt_tokens,completion_tokens,completion_text,e2e_time=aoai_call(system_message,prompt,model)
print(f"Prompt Tokens: {prompt_tokens}")
print(f"Completion Tokens: {completion_tokens}")
print(f"Total cost: ${prompt_tokens*prompt_token_cost + completion_tokens*generation_token_cost:.4f}")
print(f"Time taken: {e2e_time:.2f} seconds")


Prompt Tokens: 2004
Completion Tokens: 94
Total cost: $0.1315
Time taken: 8.20 seconds


In [7]:
completion_text

"Product Type:\nRTS\n\nSales:\nNA: 1.5M\nEU: 1.8M\nAsia: 2.0M\nLA: 0.5M\nAfrica: 0.3M\n\nPerformance:\n1.5 times initial projections.\n\nFeedback:\nOverall mixed. Criticism targeted at the game's micro-transaction system. \n\nFuture Directions:\nReevaluate the micro-transaction system and improve penetration into underperforming markets."